In [1]:
import datetime as dt
import rasterio
import numpy as np
import pandas as pd
import fiona
from shapely.geometry import mapping, shape
from shapely.geometry import box as shBox
import rasterio
# from rasterio import windows as win

### read csv file of the climate data downloaded for each year

In [2]:
# to generate the LAI, we downloaded the layer from Copernicus land service portal: https://land.copernicus.eu/global/products/lai if the download is customized the value of the layer will be DN. the number is multiplied by 1/30 to convert to LAI value
# the null values are filled with the 0.5 LAI value. The cell size of the data is 300m. We converted all layers (summer and winter) to 30m with the same projectsion to be able to use in the model. 


In [3]:
# set the year
year=2011

In [6]:
# set the csv of the climate file
d=pd.read_csv(r'G:\NationalLayer\ClimateData\Raw_CDC_data\{}csv\{}.csv'.format(year,year),header=None)
# set the columns of the csv file here
d.columns = ["Station", "Date", "PRCP_Type", "prcp","Measurement_Flag","Quality_Flag","Source_Flag","col99"]
# set the station shapefile address
stationPointShape=r'G:\NationalLayer\ClimateData\Stations\Stations_All_USAl.shp'
field_id='StationFID'
# set the reasters for the start of seson of the year
startSeasonAddress=r'G:\NationalLayer\ClimateData\Phenology\2011\SOST2011_30m_projected.img'
endSeasonAddress  =r'G:\NationalLayer\ClimateData\Phenology\2011\EOST2011_30m_projected.img'


In [7]:
d.sample(10)

,Station,Date,PRCP_Type,prcp,Measurement_Flag,Quality_Flag,Source_Flag,col99
26737527,USC00473038,20110923,SNOW,0,NaN,NaN,7,NaN
5284427,USW00003820,20110221,WDF2,250,NaN,NaN,X,NaN
9937034,USW00003195,20110407,WDF2,220,NaN,NaN,W,NaN
21671393,US1ILCK0014,20110803,PRCP,150,NaN,NaN,N,NaN
29594541,USS0022G13S,20111022,TMAX,171,NaN,NaN,T,NaN
19657443,USC00267123,20110713,TMIN,128,NaN,NaN,7,700.0
3110198,USC00139132,20110130,TMIN,-144,NaN,NaN,K,700.0
16702593,USS0013C36S,20110614,TMAX,119,NaN,NaN,T,NaN
6645619,US1NMBR0040,20110306,SNWD,0,NaN,NaN,N,NaN
26804440,USW00003996,20110923,WDF2,10,NaN,NaN,X,NaN


In [8]:
# set a radious search in meter for creating a box around each station to get the average of season end and start
search=1000

In [9]:
def findWindow (shapeBound,mainRasterBnd,mainRasterCellSize):
    startRow = int((mainRasterBnd[3] - shapeBound[3])/mainRasterCellSize)
    endRow   = int((shapeBound[3] - shapeBound[1])/mainRasterCellSize)+1+startRow
    startCol = int((shapeBound[0] - mainRasterBnd[0])/mainRasterCellSize)
    endCol   = int((shapeBound[2] - shapeBound[0])/mainRasterCellSize)+1+startCol
    return (startRow,endRow,startCol,endCol)

In [10]:
with rasterio.open(startSeasonAddress) as rstNational:
            kwds = rstNational.meta.copy()
            mainRasterBnd=rstNational.bounds
            cellSize= kwds['transform'][0]

In [11]:
# thie is a dictionary that the key is station_id and the value is [startS,endS]
dictStartEndSeacon={}
for stPoint in fiona.open(stationPointShape):
    st_id=stPoint['properties'][field_id]
    p=(shape(stPoint['geometry']))
    st_coors=(p.x,p.y)
    # (left,butt,right,top)
    searchBound=(p.x-search,p.y-search,p.x+search,p.y+search)
    # let's the bound in the national raster
    window_use=findWindow(searchBound,mainRasterBnd,cellSize)
    getWin=((window_use[0],window_use[1]),(window_use[2],window_use[3]))
    # read the windown in each raster and get the values

    with rasterio.open(startSeasonAddress) as src:
        nodata = (src.meta.copy())['nodata']
        startSeason_ar=(src.read(1, window=getWin)).astype('float')
        startSeason_ar[(startSeason_ar==nodata)|(startSeason_ar>365)|(startSeason_ar<0)]=np.nan
    with rasterio.open(endSeasonAddress) as src:
        nodata = (src.meta.copy())['nodata']
        endSeason_ar=(src.read(1, window=getWin)).astype('float')
        endSeason_ar[(endSeason_ar==nodata)|(endSeason_ar>365)|(endSeason_ar<0)]=np.nan
    if (np.nanmean(startSeason_ar)>=0)&(np.nanmean(endSeason_ar)>=0):
        stSos=np.nanmean(startSeason_ar)
        stEos=np.nanmean(endSeason_ar)

    sosDt=(dt.datetime(year, 1, 1) + dt.timedelta(stSos - 1))
    eosDt=(dt.datetime(year, 1, 1) + dt.timedelta(stEos - 1))
    dictStartEndSeacon[str(st_id)]=[sosDt.strftime('%Y-%m-%d'),eosDt.strftime('%Y-%m-%d')]

C:\ProgramData\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: Mean of empty slice


In [12]:
lstDictValues=[]
for key, val in dictStartEndSeacon.items():
    lstDictValues.append([key,val[0],val[1]])

In [13]:
dft=pd.DataFrame(lstDictValues,columns=['st_unique_ID','SOS','EOS'])
dft['SOS']=pd.to_datetime(dft['SOS'], format='%Y-%m-%d')
dft['EOS']=pd.to_datetime(dft['EOS'], format='%Y-%m-%d')
end_year = pd.Timestamp('{}-12-30'.format(year))
mask=((dft['SOS']<=end_year) & (dft['EOS'] <= end_year)&(dft['SOS']<=dft['EOS']))
dft=dft[mask]
dft['st_unique_ID']=dft['st_unique_ID'].astype('int')

In [14]:
dft.sample(3)

,st_unique_ID,SOS,EOS
18840,18841,2011-04-11,2011-11-01
3195,3196,2011-01-09,2011-05-23
52244,52245,2011-04-16,2011-10-24


In [15]:
dft.to_csv(r'G:\NationalLayer\ClimateData\DailySummary\Station_Season_{}.csv'.format(year))

In [16]:
dfCSV_US= pd.read_csv(r'G:\NationalLayer\ClimateData\Stations\Stations_All_USA.csv')
dfCSV_US['StationFID']=dfCSV_US['StationFID'].astype('int')

In [17]:
dft.head(2)

,st_unique_ID,SOS,EOS
0,1,2011-04-27,2011-11-25
1,2,2011-02-24,2011-10-25


In [18]:
dfCSV_US.rename(columns={'StationFID':'st_unique_ID_USCSV'}, inplace=True)

In [19]:
dfCSV_US.head(3)

,Station_ID,Ycoor,Xcoor,Elevation,Name,col2,col3,col4,col5,col6,col7,col8,col9,col10,st_unique_ID_USCSV
0,CA001018611,48.0333,-123.3333,70.0,BC,VICTORIA,GONZALES,CS,71200,0,NaN,NaN,NaN,NaN,1
1,CA001102420,49.0000,-123.0833,51.0,BC,DELTA,PEBBLE,HILL,NaN,0,NaN,NaN,NaN,NaN,2
2,CA001103635,49.0000,-122.2167,8.0,BC,HUNTINGDON,METER,STATION,NaN,0,NaN,NaN,NaN,NaN,3


In [20]:
dfSeasons=dfCSV_US.set_index('st_unique_ID_USCSV').join(dft.set_index('st_unique_ID'))

In [21]:
dfSeasons['st_unique_ID']=(dfSeasons.index).astype('int')
dfSeasons.sample(10)

,Station_ID,Ycoor,Xcoor,Elevation,Name,col2,col3,col4,col5,col6,col7,col8,col9,col10,SOS,EOS,st_unique_ID
st_unique_ID_USCSV,,,,,,,,,,,,,,,,,
40667,USC00194075,42.5500,-73.2333,376.7,MA,LANESBORO,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,2011-04-15,2011-10-23,40667
44802,USC00293565,34.1167,-104.3000,1064.7,NM,GLEN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,2011-06-18,2011-11-18,44802
55021,USR0000INEA,41.5667,-93.2583,273.7,IA,NEAL,SMITH,IOWA,NaN,0,NaN,NaN,NaN,NaN,2011-04-30,2011-10-17,55021
25161,US1SCCR0104,32.8415,-79.8233,4.9,SC,MOUNT,PLEASANT,2.7,ENE,0,NaN,NaN,NaN,NaN,2011-05-07,2011-11-18,25161
5451,US1COFM0076,38.3242,-105.0433,1677.9,CO,FLORENCE,5.8,SE,NaN,0,NaN,NaN,NaN,NaN,2011-05-11,2011-09-29,5451
39921,USC00165890,31.5667,-93.4833,70.1,LA,MANY,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,2011-03-27,2011-10-14,39921
5096,US1CODN0056,39.6576,-104.8858,1687.1,CO,CHERRY,CREEK,DAM,1.7,0,NaN,NaN,NaN,NaN,2011-04-27,2011-11-21,5096
32870,US1WYJN0007,43.7061,-106.2982,1348.1,WY,KAYCEE,17,E,NaN,0,NaN,NaN,NaN,NaN,2011-04-08,2011-11-12,32870
37008,USC00101932,45.0833,-114.2333,1526.4,ID,COBALT,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,2011-04-24,2011-11-14,37008


In [22]:
intcptRate_on  = 0.3
intcptRate_off = 0.1
storageCap_off = 1
storageCap_on  = 3

### read the stations' csv file. This could be geographically selected. They also come with and StationFID col

#### list of states

In [23]:
lststates = ["AL", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [24]:
d.sample(5)

,Station,Date,PRCP_Type,prcp,Measurement_Flag,Quality_Flag,Source_Flag,col99
36320587,PKM00041715,20111228,TAVG,143,H,NaN,S,NaN
24804496,US1AZPM0093,20110903,PRCP,3,NaN,NaN,N,NaN
24446907,US1TXZV0011,20110831,SNOW,0,NaN,NaN,N,NaN
17607648,SWE00139330,20110623,SNWD,0,NaN,NaN,E,NaN
11431063,NOE00134226,20110422,SNWD,480,NaN,NaN,E,NaN


### join the climate data dframe to the stations' dframe and generate a df that carries all cols

In [25]:
df=d.set_index('Station').join(dfSeasons.set_index('Station_ID'))
df.rename(columns={'Name': 'State', 'col2': 'Place'}, inplace=True)

In [26]:
df['NOAA_st_ID']=df.index

In [27]:
df.sample(2)

,Date,PRCP_Type,prcp,Measurement_Flag,Quality_Flag,Source_Flag,col99,Ycoor,Xcoor,Elevation,...,col5,col6,col7,col8,col9,col10,SOS,EOS,st_unique_ID,NOAA_st_ID
ASN00010007,20111014,PRCP,0,NaN,NaN,a,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,ASN00010007
USW00014891,20111113,WT16,1,NaN,NaN,X,NaN,40.8203,-82.5178,394.7,...,AP,72420.0,NaN,NaN,NaN,NaN,2011-04-12,2011-11-17,57197.0,USW00014891


In [28]:
df.columns

Index(['Date', 'PRCP_Type', 'prcp', 'Measurement_Flag', 'Quality_Flag',
       'Source_Flag', 'col99', 'Ycoor', 'Xcoor', 'Elevation', 'State', 'Place',
       'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'SOS',
       'EOS', 'st_unique_ID', 'NOAA_st_ID'],
      dtype='object')

In [29]:
# create a list of all places of the df
lstAll=df['State']
# select only rows that their 'state' col is in the list of our states
dfUS=df[lstAll.isin(lststates)]

In [30]:
dfUS['st_unique_ID']=(dfUS['st_unique_ID']).astype('int')
dfUS.head(4)

C:\ProgramData\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,PRCP_Type,prcp,Measurement_Flag,Quality_Flag,Source_Flag,col99,Ycoor,Xcoor,Elevation,...,col5,col6,col7,col8,col9,col10,SOS,EOS,st_unique_ID,NOAA_st_ID
US009052008,20110101,TMAX,-142,NaN,NaN,C,NaN,43.7333,-96.6333,482.0,...,CANADA),0.0,NaN,NaN,NaN,NaN,2011-04-18,2011-10-26,17,US009052008
US009052008,20110101,TMIN,-204,NaN,NaN,C,NaN,43.7333,-96.6333,482.0,...,CANADA),0.0,NaN,NaN,NaN,NaN,2011-04-18,2011-10-26,17,US009052008
US009052008,20110101,PRCP,0,NaN,NaN,C,NaN,43.7333,-96.6333,482.0,...,CANADA),0.0,NaN,NaN,NaN,NaN,2011-04-18,2011-10-26,17,US009052008
US009052008,20110101,SNWD,50,NaN,NaN,C,NaN,43.7333,-96.6333,482.0,...,CANADA),0.0,NaN,NaN,NaN,NaN,2011-04-18,2011-10-26,17,US009052008


In [31]:
dfUS=dfUS[(dfUS['PRCP_Type']=='PRCP')]

In [32]:
# select only rows that have some rainfall (type and amount>0)
dfUS=dfUS[(dfUS['prcp']>0)]
len(dfUS.prcp)

1876603

In [33]:
# convert the date col to a date object
lstDates=dfUS['Date']
dfUS['Date']=pd.to_datetime(dfUS['Date'], format='%Y%m%d')

In [34]:
dfUS.sample(10)

,Date,PRCP_Type,prcp,Measurement_Flag,Quality_Flag,Source_Flag,col99,Ycoor,Xcoor,Elevation,...,col5,col6,col7,col8,col9,col10,SOS,EOS,st_unique_ID,NOAA_st_ID
US1ILHY0005,2011-04-24,PRCP,3,NaN,NaN,N,NaN,41.4334,-90.4175,224.0,...,SE,0.0,NaN,NaN,NaN,NaN,2011-04-05,2011-10-25,10748,US1ILHY0005
USC00253365,2011-04-19,PRCP,56,NaN,NaN,K,1900.0,40.9394,-100.1514,793.4,...,NaN,0.0,NaN,NaN,NaN,NaN,2011-04-22,2011-11-03,43530,USC00253365
US1ORMT0007,2011-03-23,PRCP,5,NaN,NaN,N,NaN,45.4642,-122.6984,117.7,...,ESE,0.0,NaN,NaN,NaN,NaN,2011-04-03,2011-12-28,24110,US1ORMT0007
US1NJSM0020,2011-08-15,PRCP,650,NaN,NaN,N,NaN,40.4310,-74.6548,29.9,...,ENE,0.0,NaN,NaN,NaN,NaN,2011-04-21,2011-10-04,20498,US1NJSM0020
USC00126830,2011-04-29,PRCP,3,NaN,NaN,K,900.0,40.0739,-87.5067,186.2,...,NaN,0.0,NaN,NaN,NaN,NaN,2011-04-09,2011-10-10,38096,USC00126830
USC00384581,2011-12-17,PRCP,43,NaN,NaN,7,800.0,34.9831,-83.0678,762.0,...,NaN,0.0,NaN,NaN,NaN,NaN,2011-04-22,2011-10-03,49168,USC00384581
US1CASK0002,2011-11-24,PRCP,30,NaN,NaN,N,NaN,41.6651,-122.6208,895.2,...,NaN,0.0,NaN,NaN,NaN,NaN,2011-03-22,2011-08-11,3858,US1CASK0002
USC00238746,2011-04-27,PRCP,15,NaN,NaN,7,700.0,38.5417,-90.9753,148.4,...,NaN,0.0,NaN,NaN,NaN,NaN,2011-04-06,2011-10-24,42621,USC00238746
USS0005K26S,2011-02-26,PRCP,25,NaN,NaN,T,NaN,39.5700,-105.8000,3340.6,...,NaN,0.0,NaN,NaN,NaN,NaN,2011-06-09,2011-11-03,55766,USS0005K26S
US1FLOK0029,2011-12-27,PRCP,229,NaN,NaN,N,NaN,30.4109,-86.6475,4.9,...,E,0.0,NaN,NaN,NaN,NaN,2011-08-16,2011-11-22,8598,US1FLOK0029


In [35]:
dfUS[(dfUS.index=='USC00088051') & (dfUS.Date=='2016-10-07')]

,Date,PRCP_Type,prcp,Measurement_Flag,Quality_Flag,Source_Flag,col99,Ycoor,Xcoor,Elevation,...,col5,col6,col7,col8,col9,col10,SOS,EOS,st_unique_ID,NOAA_st_ID


In [36]:
# convert the prcp col to float. Also convert that to m. The native unit of data is 10mm; that's why we divided by 10,000
dfUS['prcp'] = dfUS.prcp.astype(float)
dfUS['prcp'] = (dfUS['prcp'])/10000.0

In [37]:
dfUS['leaf_on']=0

In [38]:
dfUS.loc[(dfUS['Date'] >= dfUS['SOS'])&(dfUS['Date'] <= dfUS['EOS']), 'leaf_on'] = 1

In [39]:
dfUS.sample(2)

,Date,PRCP_Type,prcp,Measurement_Flag,Quality_Flag,Source_Flag,col99,Ycoor,Xcoor,Elevation,...,col6,col7,col8,col9,col10,SOS,EOS,st_unique_ID,NOAA_st_ID,leaf_on
US1NYUL0008,2011-06-11,PRCP,0.0107,NaN,NaN,N,NaN,41.9162,-74.1832,154.8,...,0.0,NaN,NaN,NaN,NaN,2011-05-06,2011-10-24,22500,US1NYUL0008,1
US1NCWK0009,2011-05-23,PRCP,0.0058,NaN,NaN,N,NaN,35.9114,-78.6058,122.8,...,0.0,NaN,NaN,NaN,NaN,2011-04-23,2011-11-03,19361,US1NCWK0009,1


In [40]:
# dfUS['intercepted']=0
# dfUS.loc[dfUS.leaf_on==1, 'intercepted'] = ((dfUS.loc[dfUS.leaf_on==1])['prcp'])*intcptRate_on
# dfUS.loc[dfUS.leaf_on==0, 'intercepted'] = ((dfUS.loc[dfUS.leaf_on==0])['prcp'])*intcptRate_off


# dfUS.loc[((dfUS.leaf_on==0) & (dfUS['intercepted'] >storageCap_off)), 'intercepted'] =storageCap_off
# dfUS.loc[((dfUS.leaf_on==1) & (dfUS['intercepted'] >storageCap_on)), 'intercepted'] =storageCap_on

In [41]:
dfUS.to_csv(r'G:\NationalLayer\ClimateData\DailySummary\dfUS{}_processed.csv'.format(year))